In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
file = 'oliver twist.txt'
raw_data = open(file,'r',encoding = 'utf-8').read()
raw_data = raw_data.lower()

In [3]:
raw_data[1300:1350]

'ngles with new associates. going to a funeral for '

In [4]:
print(len(raw_data))

913035


In [8]:
vocab = sorted(set(raw_data))
chartoint = {u:i for i ,u in enumerate(vocab)}
inttochar = np.array(vocab)
text2int = np.array([chartoint[c] for c in raw_data])


In [9]:
len(chartoint)

64

In [20]:
print('Unique/Special characters')
print('\n')
for char,_ in zip(chartoint,range(64)):
    print('{:4s} {:3d}'.format(repr(char),chartoint[char]))

Unique/Special characters


'\n'   0
' '    1
'!'    2
'"'    3
'#'    4
'$'    5
'%'    6
"'"    7
'('    8
')'    9
'*'   10
','   11
'-'   12
'.'   13
'/'   14
'0'   15
'1'   16
'2'   17
'3'   18
'4'   19
'5'   20
'6'   21
'7'   22
'8'   23
'9'   24
':'   25
';'   26
'?'   27
'@'   28
'['   29
']'   30
'_'   31
'a'   32
'b'   33
'c'   34
'd'   35
'e'   36
'f'   37
'g'   38
'h'   39
'i'   40
'j'   41
'k'   42
'l'   43
'm'   44
'n'   45
'o'   46
'p'   47
'q'   48
'r'   49
's'   50
't'   51
'u'   52
'v'   53
'w'   54
'x'   55
'y'   56
'z'   57
'—'   58
'‘'   59
'’'   60
'“'   61
'”'   62
'\ufeff'  63


In [26]:
print(raw_data[100:300],'\n\n----chartoint visualization----\n\n',text2int[100:300])

e anywhere in the united states and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  you may copy it, give it away or re-use it under the terms of
the project gute 

----chartoint visualization----

 [36  1 32 45 56 54 39 36 49 36  1 40 45  1 51 39 36  1 52 45 40 51 36 35
  1 50 51 32 51 36 50  1 32 45 35  1 44 46 50 51  0 46 51 39 36 49  1 47
 32 49 51 50  1 46 37  1 51 39 36  1 54 46 49 43 35  1 32 51  1 45 46  1
 34 46 50 51  1 32 45 35  1 54 40 51 39  1 32 43 44 46 50 51  1 45 46  1
 49 36 50 51 49 40 34 51 40 46 45 50  0 54 39 32 51 50 46 36 53 36 49 13
  1  1 56 46 52  1 44 32 56  1 34 46 47 56  1 40 51 11  1 38 40 53 36  1
 40 51  1 32 54 32 56  1 46 49  1 49 36 12 52 50 36  1 40 51  1 52 45 35
 36 49  1 51 39 36  1 51 36 49 44 50  1 46 37  0 51 39 36  1 47 49 46 41
 36 34 51  1 38 52 51 36]


In [31]:
import tensorflow

sq_len = 100
example_epochs = len(raw_data) // (sq_len +1)
chardataset = tensorflow.data.Dataset.from_tensor_slices(text2int)


In [32]:
for c in chardataset.take(10):
    print(inttochar[c.numpy()])

﻿
t
h
e
 
p
r
o
j
e


In [34]:
sequences = chardataset.batch(sq_len+1, drop_remainder = True)
for i in sequences.take(10):
    print(repr(''.join(inttochar[i.numpy()])) )
    print('------'*20)

'\ufeffthe project gutenberg ebook of oliver twist, by charles dickens\n\nthis ebook is for the use of anyone'
------------------------------------------------------------------------------------------------------------------------
' anywhere in the united states and most\nother parts of the world at no cost and with almost no restri'
------------------------------------------------------------------------------------------------------------------------
'ctions\nwhatsoever.  you may copy it, give it away or re-use it under the terms of\nthe project gutenbe'
------------------------------------------------------------------------------------------------------------------------
'rg license included with this ebook or online at\nwww.gutenberg.org.  if you are not located in the un'
------------------------------------------------------------------------------------------------------------------------
"ited states, you'll have\nto check the laws of the country where you are located before 

In [35]:
def split_input_tar(data):
    input_ = data[:-1]
    target = data[1:]
    return input_ , target
dataset = sequences.map(split_input_tar)

In [37]:
for input_ex , target_ex in dataset.take(1):
    print(repr(''.join(inttochar[input_ex.numpy()])))
    print(repr(''.join(inttochar[target_ex.numpy()])))

'\ufeffthe project gutenberg ebook of oliver twist, by charles dickens\n\nthis ebook is for the use of anyon'
'the project gutenberg ebook of oliver twist, by charles dickens\n\nthis ebook is for the use of anyone'


In [38]:
BATCH_SIZE = 30
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [39]:
print('Shape {}'.format(dataset))

Shape <BatchDataset shapes: ((30, 100), (30, 100)), types: (tf.int32, tf.int32)>


In [69]:
from tensorflow.keras.layers import Dense, GRU, Dropout, Embedding
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(len(vocab),256,batch_input_shape= [BATCH_SIZE,None]))
model.add(GRU(1024, return_sequences = 'True', stateful = True, recurrent_initializer = 'glorot_uniform'))
model.add(GRU(128, return_sequences = 'True', stateful = True, recurrent_initializer = 'glorot_uniform'))
model.add(Dense(len(vocab)))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (30, None, 256)           16384     
_________________________________________________________________
gru_4 (GRU)                  (30, None, 1024)          3938304   
_________________________________________________________________
gru_5 (GRU)                  (30, None, 128)           443136    
_________________________________________________________________
dense_2 (Dense)              (30, None, 64)            8256      
Total params: 4,406,080
Trainable params: 4,406,080
Non-trainable params: 0
_________________________________________________________________


In [70]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
loss = SparseCategoricalCrossentropy(from_logits = True, )

model.compile(optimizer = 'adam', loss = loss)


In [74]:
import os, datetime
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
path = r'Predictive text generation/training_checkpoints'
file = os.path.join(path,"ckpt_{epoch}")

checkpoint = ModelCheckpoint(filepath = file, save_weights_only = True)

In [75]:
logdr = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = TensorBoard(logdr, histogram_freq = 1)

In [76]:
history = model.fit(dataset, epochs = 64, callbacks = [checkpoint,tb])

Epoch 1/64
301/301 [==============================] - 394s 1s/step - loss: 2.1934
Epoch 2/64
301/301 [==============================] - 401s 1s/step - loss: 1.8820
Epoch 3/64
301/301 [==============================] - 381s 1s/step - loss: 1.6936
Epoch 4/64
301/301 [==============================] - 398s 1s/step - loss: 1.5707
Epoch 5/64
301/301 [==============================] - 3218s 11s/step - loss: 1.4862
Epoch 6/64
301/301 [==============================] - 389s 1s/step - loss: 1.4219
Epoch 7/64
301/301 [==============================] - 386s 1s/step - loss: 1.3721
Epoch 8/64
301/301 [==============================] - 383s 1s/step - loss: 1.3332
Epoch 9/64
301/301 [==============================] - 384s 1s/step - loss: 1.2977
Epoch 10/64
301/301 [==============================] - 385s 1s/step - loss: 1.2678
Epoch 11/64
301/301 [==============================] - 391s 1s/step - loss: 1.2404
Epoch 12/64
301/301 [==============================] - 393s 1s/step - loss: 1.2140
Epoch 13/64

In [83]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [85]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 46540), started 0:03:52 ago. (Use '!kill 46540' to kill it.)

In [89]:
tensorflow.train.latest_checkpoint(path)

'Predictive text generation/training_checkpoints\\ckpt_64'

In [91]:
def build_model():
    model = Sequential()
    model.add(Embedding(len(vocab),256,batch_input_shape= [1,None]))
    model.add(GRU(1024, return_sequences = 'True', stateful = True, recurrent_initializer = 'glorot_uniform'))
    model.add(GRU(128, return_sequences = 'True', stateful = True, recurrent_initializer = 'glorot_uniform'))
    model.add(Dense(len(vocab)))
    model.summary()
    return model

In [92]:
model = build_model()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16384     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 128)            443136    
_________________________________________________________________
dense_3 (Dense)              (1, None, 64)             8256      
Total params: 4,406,080
Trainable params: 4,406,080
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.load_weights(tensorflow.train.latest_checkpoint(path))
model.build(tensorflow.TensorShape([1,None]))

In [94]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16384     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 128)            443136    
_________________________________________________________________
dense_3 (Dense)              (1, None, 64)             8256      
Total params: 4,406,080
Trainable params: 4,406,080
Non-trainable params: 0
_________________________________________________________________


In [102]:
def generate_text(model, context, num_generate = 1000, temperature = 1.0):
    input_eval = [chartoint[c] for c in context]
    input_eval = tensorflow.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tensorflow.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tensorflow.random.categorical(predictions, num_samples = 1)[-1,0].numpy()
        input_eval = tensorflow.expand_dims([predicted_id],0)
        text_generated.append(inttochar[predicted_id])
        
    return (context + ''.join(text_generated))
        
        

In [103]:
print(generate_text(model,"the tea was made", num_generate=1000, temperature=1.0))

the tea was made, resounding; seeping
into a state of the parlry, “you did she got a fixed that you have occum sometimes to, the
book-night. at such drew from a lives of anifore the moment he
had got been forght” said the
girl after a good further pipe, oliver and his new moments.
innors what more left the matter. won’t scream of hig? i’ll go _lier? my dear. ye had the grat stances of beats
malicious coughho the gentleman that new
the orphan his hand who have hit with all his recistables than to this work.. “well, well give me; i’ve knowswas done, there so much so i don’t know. i don’t
know what all, i don’t keep of this work. “if the what?” exclocked the jew.

monks well ily full of paragning about the windows of those who must done for it, there’s no deny
charge.”

“what bolted, mrs. mann?” remained mrs. maylie.

mr. brownlow in the dock; thinking, having lingered a old jew in
the space. oliver “he was brought up the ttrance to such a
light at doors.

it was so much better the gentle

In [112]:
print(generate_text(model,"if the matron had moved her chair to the right she would have to the left", num_generate=200, temperature=0.8))

if the matron had moved her chair to the right she would have to the left,
which count-temper. he took stealthily availly by love a long suppredent heart to the smallest cork over the nex tender
who seemed the three medicine me; and that we led the company who
turn it into
